In [14]:
# import
import importlib
import pandas as pd
from pipeline_executor import PipelineExecutor
import nlp_analysis.word_wizard as ww

topic = "'quantumcomputing'AND'research'"

pipeline_executor = PipelineExecutor()
quantum_df = pipeline_executor.execute(query=topic, max_articles=20, overwrite=True)

Getting news article info: 100%|██████████| 58/58 [00:36<00:00,  1.61it/s]


In [64]:
importlib.reload(ww)
wizard = ww.WordWizard(df=quantum_df)
wizard.create_word_embeddings(column='body', lean=True)
wizard.cluster_embeddings('body', k=3)
wizard.summarize_medoids('body')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Creating word embeddings for:body: 100%|██████████| 28/28 [00:59<00:00,  2.12s/it]
/Users/florian/Documents/Learning/MiBA/classes/term3/Capstone/MIBA-2023-CA

Optimal K: 3


In [38]:
# manual impelementation of summarization
import transformers
from transformers import BertTokenizer
summarizer = transformers.pipeline("summarization", model="facebook/bart-large-cnn")
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

self = wizard
column = 'body'
MEDOID_SUFFIX = '_medoids'
SUMMARY_SUFFIX = '_summaries'
medoids = self.df[self.df[column + '_word_embeddings_clusters' + self.MEDOID_SUFFIX] == True] # body_word_embeddings_clusters_medoids
self.df[column + SUMMARY_SUFFIX] = None
for pos in medoids.index:
    entry = medoids.loc[pos, column]
    if type(entry) != str:
        summary = 'summarized entry was no string'
    tokenized_input = tokenizer.tokenize(entry)
    if len(tokenized_input) > 1023: # max input size for BART
        entry = ' '.join(tokenized_input[:1000]).replace(' ##', '')
    try:
        summary = summarizer(entry) # set max size (! >30)

    except:
        summary = 'could not summarize'

    if type(summary) == list:
        summary = summary[0]['summary_text']

    if type(summary) == dict:
        summary = summary['summary_text']

    self.df.loc[pos, column + SUMMARY_SUFFIX] = summary

In [65]:
import pprint as pp

for i in wizard.df['body_word_embeddings_clusters'].unique():
    print('Cluster = ', i)
    for value in medoids[medoids['body_word_embeddings_clusters'] == i]['body_summaries'].values:
        pp.pprint(value)

Cluster =  0.0
'could not summarize'
('IBM just completed its yearly Quantum Summit. The theme of the summit was “ '
 'The Next Wave , ” as IBM believes quantum is rapidly approaching an '
 'inflection point. The key to quantum computing acceptance and maturing as an '
 'industry will be building a healthy ecosystem and set of partnerships. IBM '
 'made 12 major announcements at the summit.')
('Research on flight control systems for UAS focuses on the development of '
 'resource - aware fault - tolerant algorithms and their evaluation through '
 'flight testing on UAS. Research in UAS and Flight Control Systems is an '
 'integral part of the AI @ SLU Big Idea Initiative. A biomaterial is any '
 'substance that has been engineered to interact with biological systems in a '
 'medical application.')
Cluster =  nan
Cluster =  2.0
('Quantum computing is an approach to computation that takes advantage of the '
 'quantum properties of materials and particles that dominate at small scales. '
 

In [50]:
import pprint as pp
medoids = self.df[self.df[column + '_word_embeddings_clusters' + self.MEDOID_SUFFIX] == True] # body_word_embeddings_clusters_medoids

medoids[medoids['body_word_embeddings_clusters'] == 1]['body_summaries']

254    IBM researchers say they have devised a method...
304    IBM to upgrade its full fleet of IBM Quantum s...
327    IBM to upgrade its full fleet of IBM Quantum s...
Name: body_summaries, dtype: object